In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [ ]:
# Some parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 1, 0])              # Boundary condition function coefficients
dtype = torch.float                  # Tensor data type
supervised_samples = {n for n in range(16)}
unsupervised_samples = {n for n in range(16, 272)}
dim_z = 10

In [ ]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [ ]:
#Specify right hand side and stiffness matrix
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
K = PoissonFEM.StiffnessMatrix(mesh)
rhs.set_rhs_stencil(mesh, K)

In [ ]:
trainingData = dta.StokesData(supervised_samples, unsupervised_samples)
trainingData.read_data()
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [ ]:
# define rom
rom = ROM.ROM(mesh, K, rhs, trainingData.output_resolution**2)

In [ ]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z=dim_z)

In [ ]:
# model.save()
# loaded_model = gs.GenerativeSurrogate()
# loaded_model.load()

In [ ]:
for n in range(model.data.n_supervised_samples):
    print('sample == ', n)
    model.log_lambdac_mean[n].max_iter = 3e4
    model.log_lambdac_mean[n].converge(model, model.data.n_supervised_samples, mode=n)

In [ ]:
model.fit(n_steps=5000, with_precisions=False, z_iterations=10, thetac_iterations=10000, thetaf_iterations=10, lambdac_iterations=10)

In [ ]:
model.fit(n_steps=10000, z_iterations=50, with_precisions=True, thetaf_iterations=200, thetac_iterations=500, lambdac_iterations=100)

In [ ]:
model.tauc

In [ ]:
model.z_mean

In [ ]:
# model = gs.GenerativeSurrogate()
# model.load()

In [ ]:
test_samples = {n for n in range(0, 4)}
testData = dta.StokesData(unsupervised_samples=test_samples)
testData.read_data()
# trainingData.plotMicrostruct(1)
testData.reshape_microstructure_image()

In [ ]:
uf_pred, Z, grad_norm = model.predict(testData, max_iterations=3000, optimizer='SGD', lr=1e-3)

In [ ]:
fig = plt.figure()
plt.plot(grad_norm)
plt.yscale('log')
plt.grid()
ax = plt.gca()
ax.set_ylim(1e1, 1e5)

In [ ]:
plt.xlabel('iteration')
plt.ylabel('gradient norm')
plt.title('Gradient norm w.r.t. z in prediction inference')

In [ ]:
grad_norm[-10:]

In [ ]:
model.z_mean[:4, :].data

In [ ]:
Z.data

In [ ]:
uf_pred

In [ ]:
model.data.P[:4, :]

In [ ]:
model.fit(n_steps=10)

In [ ]:
x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(x, y, np.reshape(uf_pred[3], (129, 129)),cmap='viridis', edgecolor='none')

In [ ]:
pp = np.reshape(model.data.P[3, :], (129, 129))

In [ ]:
ax.plot_surface(x, y, pp.detach().numpy())

In [ ]:
lg_lambdac = model.pcNet(Z)

In [ ]:
lg_lambdac

In [ ]:
lg_lambdac2 = model.pcNet(model.z_mean[:4, :])
lg_lambdac2

In [ ]:
model.log_lambdac_mean_tensor[:4, :]

In [ ]:
uf_pred2 = model.rom_autograd(torch.exp(lg_lambdac2[0]))

In [ ]:
x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(x, y, np.reshape(uf_pred2.detach().numpy(), (129, 129)),cmap='viridis', edgecolor='none')

In [ ]:
model.log_lambdac_mean_tensor[:4, :]

In [ ]:
fig = plt.figure()
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()

x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose

ax = []
for i in range(4):
    ax.append(fig.add_subplot(2, 4, i + 1, projection='3d'))
    ax[i].plot_surface(x, y, np.reshape(uf_pred[i], (129, 129)), edgecolor='none')
    ax[i].plot_surface(x, y, np.reshape(model.data.P[i, :], (129, 129)).detach().numpy(),
                       edgecolor='none', cmap='inferno')
    ax[i].set_position([0.025 + .2375*i, 0.55, 0.22, 0.4])
    ax[i].elev = 10
    ax[i].set_title('pressure field prediction')
    
xx = np.array([i for i in range(dim_z)])
width = 0.42  # the width of the bars
for i in range(4):
    ax.append(fig.add_subplot(2, 4, i + 5))
    ax[i + 4].bar(xx-width/2, Z[i, :].detach().numpy(), width, label='prediction')
    ax[i + 4].bar(xx + width/2, model.z_mean[i, :].detach().numpy(), width, label='training')
    ax[i + 4].set_position([0.05 + .235*i, 0.08, 0.20, 0.4])
    ax[i + 4].set_title('location of z distribution')
    ax[i + 4].set_xlabel('component')
    ax[i + 4].set_ylabel('z')
    ax[i + 4].legend()

In [ ]:
lg_lambdac2 = model.pcNet(model.z_mean[:4, :])
# lg_lambdac2 = model.log_lambdac_mean_tensor[:4, :]
uf_pred2 = []
fig = plt.figure()
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()

x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose
ax = []
for i in range(4):
    uf_pred2.append(model.rom_autograd(torch.exp(lg_lambdac2[i])))
    ax.append(fig.add_subplot(1, 4, i + 1, projection='3d'))
    ax[i].plot_surface(x, y, np.reshape(uf_pred2[i].detach().numpy(), (129, 129)), edgecolor='none')
    ax[i].plot_surface(x, y, np.reshape(model.data.P[i, :], (129, 129)).detach().numpy(),
                       edgecolor='none', cmap='inferno')
    ax[i].set_position([0.025 + .2375*i, 0.55, 0.22, 0.4])
    ax[i].elev = 10
    ax[i].set_title('pressure field prediction')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.plot_surface(x, y, np.reshape(uf_pred2[i].detach().numpy(), (129, 129)), edgecolor='none')

In [ ]:
plt.close('all')

In [ ]:
ax[0].get_position()

In [ ]:
ax[0].set_position([0.02, 0.3, 0.2, 0.4])

In [ ]:
ax[0].elev = 10

In [ ]:
ax[0].get_zticks()

In [ ]:
ax[5].set_xlabel

In [ ]:
a

In [ ]:
fc = torch.nn.Linear(5, 2)

In [ ]:
fc.weight.shape

In [ ]:
fc.bias

In [ ]:
model.training_iterations = 0

In [ ]:
from torch.distributions import constraints
from torch.distributions.exp_family import ExponentialFamily
from torch.distributions.utils import _standard_normal, broadcast_all

In [ ]:
dist = torch.distributions.Normal(0, 1)

In [ ]:
dist.stddev = 2

In [ ]:
x = torch.randn(3, 4)

In [ ]:
x

In [ ]:
sum(x)

In [2]:
def log_emp_dist(x):
    sigma_e = 2*torch.ones(1)
    mu_e = -3*torch.ones(1)
    return .5*(1/sigma_e**2)*(x - mu_e)**2

def log_emp_dist_grad(x):
    sigma_e = 2*torch.ones(1)
    mu_e = -3*torch.ones(1)
    return (x - mu_e)/sigma_e**2
    

In [3]:
from VI import variationalinference as vi
svi = vi.DiagGaussianSVI(log_emp_dist, log_emp_dist_grad, 1)

In [4]:
svi.fit()

In [ ]:
for i in range(10000):
    loss = elbo.autograd_elbo(elbo.vi_mean, elbo.vi_log_std)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

In [ ]:
print('sigma == ', elbo.vi_std)
print('mu == ', elbo.vi_mean)

In [6]:
svi.vi_std

tensor([1.9628], grad_fn=<ExpBackward>)

In [ ]:
log_sigma.grad

In [ ]:
torch.zeros(5) - 1

In [5]:
svi.vi_mean

tensor([-2.9550], requires_grad=True)